In [1]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [3]:
df_info = pd.read_csv('./Data/Preprocessing/Final_Patent_Info_dropna.csv')
df_claim = pd.read_csv('./Data/Preprocessing/Claim.csv')
df_cpc = pd.read_csv('./Data/Preprocessing/CPC.csv')
df_abstract = pd.read_csv('./Data/Preprocessing/Abstract.csv')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_ipc = pd.read_csv('./Data/Preprocessing/IPC.csv')

In [4]:
df_info.columns

Index(['출원번호', '출원일자', '발명의명칭(국문)'], dtype='object')

In [16]:
df = pd.merge(df_info, df_abstract, how= 'outer', on= '출원번호')
df = pd.merge(df, df_claim, how= 'outer', on= '출원번호')
df = pd.merge(df, df_cpc, how= 'outer', on= '출원번호')
df = pd.merge(df, df_ipc, how= 'outer', on= '출원번호')
df.head()

,출원번호,출원일자,발명의명칭(국문),초록,preprocess_abstract,청구항,cpc코드,ipc코드
0,1009880005252,19880506,가스 스트림의 처리,NaN,NaN,['황화수소를 포함하는 제1가스 스트림을 제1연소대역속으로 통과시켜서 그 속에서 이...,NaN,C01B 17/04
1,1009890015702,19891030,회전활강장치,NaN,NaN,"['경사지게 형성한 활주면과, 이 활주면 위를 활주하는 활주체와, 상기 활주면에 배...",NaN,A63G 29/00
2,1009940008104,19940418,주행 로보트 전력 제어 장치,NaN,NaN,['내부 전원과 외부로부터 전력을 공급받는 외부 전원을 선택하여 주전원으로 사용할 ...,NaN,G05F 1/10
3,1011920014920,19920819,냉장고 팬모터의 속도 안정화회로 및 그 방법,NaN,NaN,"['전원전압의 변화량을 감지하도록 한 전원전압감지입력부(33)와, 상기 전원전압감지...",NaN,H02M 1/00
4,1018830001360,19830401,디지탈 정보신호 기록방법과 장치 및 기록 캐리어,NaN,NaN,['방사 펄스가 단위 기록 마크를 기록 캐리어상에 형성되도록 하는 에너지를 갖고 있...,NaN,G11B 7/00


In [17]:
df.dropna(inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3592085 entries, 13182 to 4862453
Data columns (total 8 columns):
출원번호                   int64
출원일자                   object
발명의명칭(국문)              object
초록                     object
preprocess_abstract    object
청구항                    object
cpc코드                  object
ipc코드                  object
dtypes: int64(1), object(7)
memory usage: 246.6+ MB


In [18]:
df.reset_index(drop= True, inplace= True)
df.head()

,출원번호,출원일자,발명의명칭(국문),초록,preprocess_abstract,청구항,cpc코드,ipc코드
0,1019780001915,19780623,용성인비의조립방법,<SDOAB><SUMMARY><목적>조미료 제조시에 부산물로 얻어지는 발효 폐액과 ...,조미료 제조시에 부산물로 얻어지는 발효 폐액과 인산암모니움염의 혼합 현탁액을 이용하...,['분말 용성인비를 조립함에 있어 분말 용성인비 80-95부에 조미료 폐액에 인산 ...,C05B 17/00,C05B 17/00
1,1019780003870,19781221,2'4'-디플루오로ㅡ4-히드록시ㅡ3-비페닐카복시산의제조방법,"<SDOAB><SUMMARY><목적>2',4'-디플루오로-4-아세톡시비페닐을 출발물...","2',4'-디플루오로-4-아세톡시비페닐을 출발물질로하여 2',4'-디플루오로-4-히...","[""구조식 ( I )로 표시되는 2',4'-디플루오로-4-아세톡시비페닐을 사염화에탄...",C07C 51/02,C07C 51/02
2,1019790001108,19790407,신규한구균용1치환우라실류의제조방법,<SDOAB><SUMMARY><P>&quot;내용 없음&quot;</P></SUMM...,&quot;내용 없음&quot;,"[""하기구조식(Ⅲ)으로 표시되는 화합물을 P-시멘, 테트라린, 오르토디클로로벤젠과 ...",C07D 239/54,C07D 239/54
3,1019790001504,19790511,음향재생시스템장치,<SDOAB><SUMMARY><목적>최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청...,최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청 주파수 영역에서 평탄하고 균일한 특...,"['외벽, 혼, 스피커 원추, 저음스피커, 고음스피커로 이루어지는 음향재생장치로서,...",H04R 1/2865,H04R 1/28
4,1019790002494,19790725,퍼퍼형가스차단기,<SDOAB><SUMMARY><P>내용 없음.</P></SUMMARY><ABDR><...,내용 없음.내용 없음.,['소호성 가스를 충전한 밀폐용기내에 주가동접점(15)과 주고정접점(12)을 가진 ...,H01H 33/91,H01H 33/91


In [19]:
def check_string(data):
    if '내용 없음' in data:
        return np.nan

    else:
        return data.strip()

In [20]:
df['preprocess_abstract'] = df['preprocess_abstract'].apply(check_string)

In [21]:
df.head()

,출원번호,출원일자,발명의명칭(국문),초록,preprocess_abstract,청구항,cpc코드,ipc코드
0,1019780001915,19780623,용성인비의조립방법,<SDOAB><SUMMARY><목적>조미료 제조시에 부산물로 얻어지는 발효 폐액과 ...,조미료 제조시에 부산물로 얻어지는 발효 폐액과 인산암모니움염의 혼합 현탁액을 이용하...,['분말 용성인비를 조립함에 있어 분말 용성인비 80-95부에 조미료 폐액에 인산 ...,C05B 17/00,C05B 17/00
1,1019780003870,19781221,2'4'-디플루오로ㅡ4-히드록시ㅡ3-비페닐카복시산의제조방법,"<SDOAB><SUMMARY><목적>2',4'-디플루오로-4-아세톡시비페닐을 출발물...","2',4'-디플루오로-4-아세톡시비페닐을 출발물질로하여 2',4'-디플루오로-4-히...","[""구조식 ( I )로 표시되는 2',4'-디플루오로-4-아세톡시비페닐을 사염화에탄...",C07C 51/02,C07C 51/02
2,1019790001108,19790407,신규한구균용1치환우라실류의제조방법,<SDOAB><SUMMARY><P>&quot;내용 없음&quot;</P></SUMM...,NaN,"[""하기구조식(Ⅲ)으로 표시되는 화합물을 P-시멘, 테트라린, 오르토디클로로벤젠과 ...",C07D 239/54,C07D 239/54
3,1019790001504,19790511,음향재생시스템장치,<SDOAB><SUMMARY><목적>최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청...,최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청 주파수 영역에서 평탄하고 균일한 특...,"['외벽, 혼, 스피커 원추, 저음스피커, 고음스피커로 이루어지는 음향재생장치로서,...",H04R 1/2865,H04R 1/28
4,1019790002494,19790725,퍼퍼형가스차단기,<SDOAB><SUMMARY><P>내용 없음.</P></SUMMARY><ABDR><...,NaN,['소호성 가스를 충전한 밀폐용기내에 주가동접점(15)과 주고정접점(12)을 가진 ...,H01H 33/91,H01H 33/91


In [22]:
df.dropna(inplace= True)
df.reset_index(drop= True, inplace= True)
df.head()

,출원번호,출원일자,발명의명칭(국문),초록,preprocess_abstract,청구항,cpc코드,ipc코드
0,1019780001915,19780623,용성인비의조립방법,<SDOAB><SUMMARY><목적>조미료 제조시에 부산물로 얻어지는 발효 폐액과 ...,조미료 제조시에 부산물로 얻어지는 발효 폐액과 인산암모니움염의 혼합 현탁액을 이용하...,['분말 용성인비를 조립함에 있어 분말 용성인비 80-95부에 조미료 폐액에 인산 ...,C05B 17/00,C05B 17/00
1,1019780003870,19781221,2'4'-디플루오로ㅡ4-히드록시ㅡ3-비페닐카복시산의제조방법,"<SDOAB><SUMMARY><목적>2',4'-디플루오로-4-아세톡시비페닐을 출발물...","2',4'-디플루오로-4-아세톡시비페닐을 출발물질로하여 2',4'-디플루오로-4-히...","[""구조식 ( I )로 표시되는 2',4'-디플루오로-4-아세톡시비페닐을 사염화에탄...",C07C 51/02,C07C 51/02
2,1019790001504,19790511,음향재생시스템장치,<SDOAB><SUMMARY><목적>최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청...,최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청 주파수 영역에서 평탄하고 균일한 특...,"['외벽, 혼, 스피커 원추, 저음스피커, 고음스피커로 이루어지는 음향재생장치로서,...",H04R 1/2865,H04R 1/28
3,1019790002604,19790731,비오염화장실시스템,"<SDOAB><SUMMARY>[청구범위]도면에 도시하고 본문에 상술한 바와 같이, ...","[청구범위]도면에 도시하고 본문에 상술한 바와 같이, 보울, 일정하게 유수 보...","['도면에 도시하고 본문에 상술한 바와 같이, 보울, 일정하게 유수를 보울로 공급하...",E03D 9/10,E03D 9/10
4,1019790003025,19790904,치환된W-아로일(프로피오닐또는부티릴)-L-프로린유도체의제조방법,<SDOAB><SUMMARY>[목적] 안지오텐신 전환효소를 저해함으로써 안지오텐신 ...,[목적] 안지오텐신 전환효소를 저해함으로써 안지오텐신 I이 안지오텐신 II로의 전환...,"[""구조식(Ⅱ)의 화합물과 구조식 H-R3인 티올화제의 음이온을 반응시켜 구조식(Ⅲ...",C07D 207/16,C07D 207/16


In [23]:
df['cpc코드'] = df['cpc코드'].apply(lambda x : x.strip().split(' ')[0])
df['ipc코드'] = df['ipc코드'].apply(lambda x : x.strip().split(' ')[0])

In [24]:
df.head()

,출원번호,출원일자,발명의명칭(국문),초록,preprocess_abstract,청구항,cpc코드,ipc코드
0,1019780001915,19780623,용성인비의조립방법,<SDOAB><SUMMARY><목적>조미료 제조시에 부산물로 얻어지는 발효 폐액과 ...,조미료 제조시에 부산물로 얻어지는 발효 폐액과 인산암모니움염의 혼합 현탁액을 이용하...,['분말 용성인비를 조립함에 있어 분말 용성인비 80-95부에 조미료 폐액에 인산 ...,C05B,C05B
1,1019780003870,19781221,2'4'-디플루오로ㅡ4-히드록시ㅡ3-비페닐카복시산의제조방법,"<SDOAB><SUMMARY><목적>2',4'-디플루오로-4-아세톡시비페닐을 출발물...","2',4'-디플루오로-4-아세톡시비페닐을 출발물질로하여 2',4'-디플루오로-4-히...","[""구조식 ( I )로 표시되는 2',4'-디플루오로-4-아세톡시비페닐을 사염화에탄...",C07C,C07C
2,1019790001504,19790511,음향재생시스템장치,<SDOAB><SUMMARY><목적>최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청...,최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청 주파수 영역에서 평탄하고 균일한 특...,"['외벽, 혼, 스피커 원추, 저음스피커, 고음스피커로 이루어지는 음향재생장치로서,...",H04R,H04R
3,1019790002604,19790731,비오염화장실시스템,"<SDOAB><SUMMARY>[청구범위]도면에 도시하고 본문에 상술한 바와 같이, ...","[청구범위]도면에 도시하고 본문에 상술한 바와 같이, 보울, 일정하게 유수 보...","['도면에 도시하고 본문에 상술한 바와 같이, 보울, 일정하게 유수를 보울로 공급하...",E03D,E03D
4,1019790003025,19790904,치환된W-아로일(프로피오닐또는부티릴)-L-프로린유도체의제조방법,<SDOAB><SUMMARY>[목적] 안지오텐신 전환효소를 저해함으로써 안지오텐신 ...,[목적] 안지오텐신 전환효소를 저해함으로써 안지오텐신 I이 안지오텐신 II로의 전환...,"[""구조식(Ⅱ)의 화합물과 구조식 H-R3인 티올화제의 음이온을 반응시켜 구조식(Ⅲ...",C07D,C07D


In [25]:
df.drop('초록', axis= 1, inplace= True)
df.head()

,출원번호,출원일자,발명의명칭(국문),preprocess_abstract,청구항,cpc코드,ipc코드
0,1019780001915,19780623,용성인비의조립방법,조미료 제조시에 부산물로 얻어지는 발효 폐액과 인산암모니움염의 혼합 현탁액을 이용하...,['분말 용성인비를 조립함에 있어 분말 용성인비 80-95부에 조미료 폐액에 인산 ...,C05B,C05B
1,1019780003870,19781221,2'4'-디플루오로ㅡ4-히드록시ㅡ3-비페닐카복시산의제조방법,"2',4'-디플루오로-4-아세톡시비페닐을 출발물질로하여 2',4'-디플루오로-4-히...","[""구조식 ( I )로 표시되는 2',4'-디플루오로-4-아세톡시비페닐을 사염화에탄...",C07C,C07C
2,1019790001504,19790511,음향재생시스템장치,최소의 잔향과 왜곡 현상을 나타내면서 넓은 가청 주파수 영역에서 평탄하고 균일한 특...,"['외벽, 혼, 스피커 원추, 저음스피커, 고음스피커로 이루어지는 음향재생장치로서,...",H04R,H04R
3,1019790002604,19790731,비오염화장실시스템,"[청구범위]도면에 도시하고 본문에 상술한 바와 같이, 보울, 일정하게 유수 보...","['도면에 도시하고 본문에 상술한 바와 같이, 보울, 일정하게 유수를 보울로 공급하...",E03D,E03D
4,1019790003025,19790904,치환된W-아로일(프로피오닐또는부티릴)-L-프로린유도체의제조방법,[목적] 안지오텐신 전환효소를 저해함으로써 안지오텐신 I이 안지오텐신 II로의 전환...,"[""구조식(Ⅱ)의 화합물과 구조식 H-R3인 티올화제의 음이온을 반응시켜 구조식(Ⅲ...",C07D,C07D


In [26]:
df.to_csv('./Data/Use_Data/BERT_Classification_Data.csv', index= False, encoding= 'utf-8-sig')